# IMPORTS AND FILE INPUT

In [1]:
import pandas as pd

import twint
import os
import time
import nest_asyncio
nest_asyncio.apply()
from threading import Thread

import tweepy
import pandas as pd
import numpy as np

In [2]:
os.getcwd()

'D:\\OneDrive - Indian School of Business\\Projects\\JSI\\JSI_OnlineActivity\\ISB-JSI_Project-OnlineActivity\\Tweets Codes\\Scraper'

In [3]:
os.chdir(r'D:\OneDrive - Indian School of Business\Projects\JSI\JSI_OnlineActivity\Tweets Data\input')
os.listdir()

['NSE Nifty500 List.csv', 'Updated_handles_list.csv']

In [4]:
df= pd.read_csv('Updated_handles_list.csv')
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df['CompanyName']= df['CompanyName'].str.replace('.', '')
df.to_csv('Updated_handles_list.csv')
df.shape

(332, 10)

In [5]:
df.head()

,Industry,CompanyName,cn,Symbol,Series,ISIN Code,handle,verified,descriptions,tweetscount
0,INDUSTRIAL MANUFACTURING,ABB India,abbindialtd,EQ,ABB,INE117A01022,ABBIndia,False,A #technology leader that is driving the digit...,2307
1,INDUSTRIAL MANUFACTURING,AIA Engineering,aiaengineeringltd,EQ,AIAENG,INE212H01026,aiaengineers,False,Consulting Engineering firm founded in 1985 pr...,202
2,METALS,APL Apollo Tubes,aplapollotubesltd,EQ,APLAPOLLO,INE702C01019,aplapollo_tubes,False,India's Largest Structural Steel Tube Brand,848
3,SERVICES,Adani Enterprises,adanienterprisesltd,EQ,ADANIENT,INE423A01024,AdaniOnline,True,A leading business conglomerate enriching live...,2642
4,CONSUMER GOODS,Aditya Birla Fashion and Retail,adityabirlafashionandretailltd,EQ,ABFRL,INE647O01011,AdityaBirlaGrp,True,Official Twitter account of Aditya Birla Group...,4165


# TWEEPY VALIDATION

In [6]:
# # assign the values accordingly 
# consumer_key = "OMJv2A5J1XvXdBSLykjbzQvjN" 
# consumer_secret = "IF1tUCs5uFhKNN4PILugYgTdfaFDEakS4gbzWHKEL92dIoSRHA" 
# access_token = "3097042092-HXzJwRZ4BRSiTa67RfHnlE7CeF01gutElIP6UoT" 
# access_token_secret = "kZYcycMakp0z3jcGC07UpLynOWvvtoBiAmwVjJKziSYB8"

# # authorization of consumer key and consumer secret 
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
  
# # set access to user's access key and access secret  
# auth.set_access_token(access_token, access_token_secret) 
  
# # calling the api  
# api = tweepy.API(auth, wait_on_rate_limit=True)

In [7]:
# industries=[]
# companies=[]
# cns=[]
# symbols=[]
# series=[]
# isins=[]
# handles=[]
# verified=[]
# descriptions=[]
# tweetscount=[]

# for i in range(len(df)):
#     j= df.iloc[i]['handle']
#     try:
#         j= df.iloc[i]['handle']
#         obj= api.get_user(j)
#         result=obj.verified
        
#         industries.append(df.iloc[i]['Industry'])
#         companies.append(df.iloc[i]['CompanyName'])
#         cns.append(df.iloc[i]['cn'])
#         symbols.append(df.iloc[i]['Series'])
#         series.append(df.iloc[i]['Symbol'])
#         isins.append(df.iloc[i]['ISIN Code'])
#         handles.append(j)
#         verified.append(result)
#         descriptions.append(obj.description)
#         tweetscount.append(obj.statuses_count)
#     except:
#         print('bad user name,', j)
#         pass

# columns = ['Industry', 'CompanyName', 'cn', 'Symbol', 'Series', 'ISIN Code', 'handle', 'verified', 'descriptions', 'tweetscount']
# gdf = pd.DataFrame(list(zip(industries,companies,cns,symbols,series,isins,handles,verified,descriptions,tweetscount)), columns =columns) 
# gdf.to_csv("updated_handles_tweepy_verification.csv", index=False)

In [8]:
df.verified.value_counts()

False    198
True     134
Name: verified, dtype: int64

In [9]:
d= df[df.verified==True]
d= d[d.tweetscount>100]
d.shape

(134, 10)

In [10]:
len(d[d.tweetscount>1000])

128

In [11]:
d.CompanyName.to_list()

['Adani Enterprises',
 'Aditya Birla Fashion and Retail',
 'Akzo Nobel India',
 'Bajaj Electricals',
 'Bharat Petroleumporation',
 'EIH',
 'HDFC Life Insurancepany',
 'Hindustan Petroleumporation',
 'ICICI Lombard General Insurancepany',
 'ICICI Prudential Life Insurancepany',
 'IDFC',
 'IRCON International',
 'Power Gridporation of India',
 'Procter & Gamble Health',
 'Procter & Gamble Hygiene & Health Care',
 'Rallis India',
 'SBI Cards and Payment Services',
 'SBI Life Insurancepany',
 'Schneider Electric Infrastructure',
 'Suzlon Energy',
 'TV18 Broadcast',
 'TVS Motorpany',
 'Tatamunications',
 'Tata MotorsDVR',
 'United Breweries',
 'Yes Bank',
 'Zensar Technolgies',
 '3M India',
 'AU Small Finance Bank',
 'Abbott India',
 'Aditya Birla Capital',
 'Apollo Hospitals Enterprise',
 'Apollo Tyres',
 'Asian Paints',
 'Astral Poly Technik',
 'Axis Bank',
 'BASF India',
 'BSE',
 'Bajaj Auto',
 'Bandhan Bank',
 'Bank of Baroda',
 'Bata India',
 'Bayer Cropscience',
 'Berger Paints India'

# TWINT SCRAPER

In [12]:
gdf=df

In [13]:
def tweetbyHandle(handle, company):
    c= twint.Config()
    c.Username = str(handle)
    c.Since = "2018-01-01"
    c.Until = "2020-12-31"
    c.Lang= "en"
    c.Hide_output = True
    c.Format = "Username: {username} | Tweet: {tweet}"
    c.Store_csv = True
    c.Output = str(company)+".csv"
    c.Debug = True
    
    try:
        if str(company)+".csv" in os.listdir():
            print(str(company)+".csv", " scrapped already")
            pass
        else:
            twint.run.Search(c)    
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print("Problem with %s." % since)

In [14]:
os.chdir(r'D:\OneDrive - Indian School of Business\Projects\JSI\JSI_OnlineActivity\Tweets Data\output\collected')

In [15]:
gdf.reset_index(inplace=True)
gdf.drop('index', axis=1, inplace=True)

In [ ]:
start_time= time.time()
for rows in df.itertuples():
    company = getattr(rows, 'CompanyName')##Getting the company name
    handle = getattr(rows, 'handle')
    ##Getting the Twitter handles of the required company
    print(handle + "start")
    tweetbyHandle(handle = handle, company = company)##Initialising the tweet function
    print(company + "done")
print("time taken= ", (time.time()-start_time)/60, " minutes")

ABBIndiastart
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
ABB Indiadone
aiaengineersstart
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
AIA Engineeringdone
aplapollo_tubesstart
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
APL Apollo Tubesdone
AdaniOnlinestart


CRITICAL:root:twint.output:_output:CSV:Error:[Errno 13] Permission denied: 'Adani Enterprises.csv'
CRITICAL:root:twint.output:_output:CSV:Error:[Errno 13] Permission denied: 'Adani Enterprises.csv'
CRITICAL:root:twint.output:_output:CSV:Error:[Errno 13] Permission denied: 'Adani Enterprises.csv'
CRITICAL:root:twint.output:_output:CSV:Error:[Errno 13] Permission denied: 'Adani Enterprises.csv'
CRITICAL:root:twint.output:_output:CSV:Error:[Errno 13] Permission denied: 'Adani Enterprises.csv'
CRITICAL:root:twint.output:_output:CSV:Error:[Errno 13] Permission denied: 'Adani Enterprises.csv'


[Errno 13] Permission denied: 'Adani Enterprises.csv' [x] output._output
[Errno 13] Permission denied: 'Adani Enterprises.csv' [x] output._output
[Errno 13] Permission denied: 'Adani Enterprises.csv' [x] output._output
[Errno 13] Permission denied: 'Adani Enterprises.csv' [x] output._output
[Errno 13] Permission denied: 'Adani Enterprises.csv' [x] output._output
[Errno 13] Permission denied: 'Adani Enterprises.csv' [x] output._output
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Adani Enterprisesdone
AdityaBirlaGrpstart
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Aditya Birla Fashion and Retaildone
AdvancedEnzymesstart
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Advanced Enzyme Techdone
Aegis_Logisticsstart
[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.
Aegis Logisticsdone
AjantaPharmaLtdstart
[!] No more data! Scraping will stop now.
found 0 de